<a href="https://colab.research.google.com/github/valterlucena/facial-expression-detector/blob/master/facial_expressions_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @title Dependências

import numpy as np  # manipular dados

import tensorflow as tf  # tensorflow
import tensorflow.keras as keras  
import matplotlib.pyplot as plt
import os, cv2
from google.colab.patches import cv2_imshow

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
path = '/content/gdrive/My Drive/PercepcaoComputacional/faces/'
imgfiles = [ifile for ifile in os.listdir(path) if ifile.endswith('.jpg')]
images = []
for imgfile in imgfiles:
    img = cv2.imread(path + imgfile)
    images.append(cv2.resize(img, (945,1181), interpolation = cv2.INTER_AREA))

In [0]:
cv2_imshow(images[0])

In [0]:
imagesnp = np.array(images)
imagesnp = imagesnp.reshape(-1, 945,1181, 3)

In [97]:
model = keras.Sequential([
    keras.applications.vgg16.VGG16(input_shape=images[0].shape,
                                   include_top=False),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dense(14, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 36, 29, 512)       14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 14)                14350     
Total params: 16,303,950
Trainable params: 16,303,950
Non-trainable params: 0
_________________________________________________________________
